Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [ ]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from tensorflow.contrib.learn.python.learn.datasets.mnist import read_data_sets
from tensorflow.examples.tutorials.mnist import input_data

First reload the data we generated in _notmist.ipynb_.

In [21]:

data_set = input_data.read_data_sets('', False)
training_data=data_set.train
testing_data=data_set.test

Extracting train-images-idx3-ubyte.gz
Extracting train-labels-idx1-ubyte.gz
Extracting t10k-images-idx3-ubyte.gz
Extracting t10k-labels-idx1-ubyte.gz


In [30]:

images_feed, labels_feed = training_data.next_batch(10000,False)
image_size = 28
num_labels = 10
images_feed

array([[ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       ..., 
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.]], dtype=float32)

In [29]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 2 to [0.0, 1.0, 0.0 ...], 3 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [4]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

### Logistic

In [11]:
batch_size = 128
beta=.01

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  logits = tf.matmul(tf_train_dataset, weights) + biases
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels)+beta*tf.nn.l2_loss(weights))
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf_valid_dataset, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

In [12]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = training_data.next_batch(batch_size)
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 51.175274
Minibatch accuracy: 7.0%
Validation accuracy: 9.0%
Minibatch loss at step 500: 0.976922
Minibatch accuracy: 80.5%
Validation accuracy: 80.3%
Minibatch loss at step 1000: 0.749123
Minibatch accuracy: 82.0%
Validation accuracy: 80.9%
Minibatch loss at step 1500: 0.877641
Minibatch accuracy: 76.6%
Validation accuracy: 81.0%
Minibatch loss at step 2000: 0.928107
Minibatch accuracy: 79.7%
Validation accuracy: 81.2%
Minibatch loss at step 2500: 0.830246
Minibatch accuracy: 78.9%
Validation accuracy: 81.1%
Minibatch loss at step 3000: 0.641256
Minibatch accuracy: 86.7%
Validation accuracy: 81.4%
Test accuracy: 88.4%


---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

how is this overfitting?

In [8]:
batch_size = 128
beta=.005
nHidden=1024

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights_hidden1 = tf.Variable(
    tf.truncated_normal([image_size * image_size, nHidden]))
  weights = tf.Variable(
    tf.truncated_normal([nHidden, num_labels]))
  biases_hidden1 = tf.Variable(tf.zeros([nHidden]))
  biases = tf.Variable(tf.zeros([num_labels]))
    
  # Training computation.
  hidden1 = tf.nn.relu(tf.matmul(tf_train_dataset, weights_hidden1) + biases_hidden1)
  logits = tf.matmul(hidden1, weights) + biases
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels)+beta*tf.nn.l2_loss(weights))
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset, weights_hidden1) + biases_hidden1), weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset, weights_hidden1) + biases_hidden1), weights) + biases)

In [9]:
num_steps = 101

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (1 * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 425.018555
Minibatch accuracy: 7.8%
Validation accuracy: 36.0%
Test accuracy: 85.3%


### NNet

In [40]:
batch_size = 128
beta=0.00001
nHidden=1024

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights_hidden1 = tf.Variable(
    tf.truncated_normal([image_size * image_size, nHidden]))
  weights = tf.Variable(
    tf.truncated_normal([nHidden, num_labels]))
  biases_hidden1 = tf.Variable(tf.zeros([nHidden]))
  biases = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  hidden1 = tf.nn.relu(tf.matmul(tf_train_dataset, weights_hidden1) + biases_hidden1)
  logits = tf.matmul(hidden1, weights) + biases
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))+beta*tf.nn.l2_loss(weights_hidden1)

  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset, weights_hidden1) + biases_hidden1), weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset, weights_hidden1) + biases_hidden1), weights) + biases)

In [42]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (1 * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 370.865234
Minibatch accuracy: 7.8%
Validation accuracy: 31.4%
Minibatch loss at step 500: 3.089524
Minibatch accuracy: 100.0%
Validation accuracy: 64.7%
Minibatch loss at step 1000: 3.074125
Minibatch accuracy: 100.0%
Validation accuracy: 64.7%
Minibatch loss at step 1500: 3.058794
Minibatch accuracy: 100.0%
Validation accuracy: 64.7%
Minibatch loss at step 2000: 3.043553
Minibatch accuracy: 100.0%
Validation accuracy: 64.7%
Minibatch loss at step 2500: 3.028372
Minibatch accuracy: 100.0%
Validation accuracy: 64.7%
Minibatch loss at step 3000: 3.013259
Minibatch accuracy: 100.0%
Validation accuracy: 64.7%
Test accuracy: 70.5%


---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

In [34]:
tf.nn.dropout?

In [55]:
training_data.next_batch(10)

(array([[ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        ..., 
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.]], dtype=float32),
 array([2, 1, 5, 0, 6, 9, 3, 1, 0, 9], dtype=uint8))

In [53]:
batch_size = 128
nHidden=1024
beta=0.00001

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
   
  # Variables.
  weights_hidden1 = tf.Variable(tf.truncated_normal([image_size * image_size, nHidden]))
  weights = tf.Variable(tf.truncated_normal([nHidden, image_size*image_size]))
  biases_hidden1 = tf.Variable(tf.zeros([nHidden]))
  biases = tf.Variable(tf.zeros([image_size*image_size]))
  
  # Training computation.
  hidden1 = tf.nn.dropout(tf.nn.relu(tf.matmul(tf_train_dataset, weights_hidden1) + biases_hidden1), 0.5 )
  output_units = tf.matmul(hidden1, weights) + biases
  loss =   tf.nn.l2_loss(output_units, tf_train_dataset)  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.

TypeError: Using a `tf.Tensor` as a Python `bool` is not allowed. Use `if t is not None:` instead of `if t:` to test if a tensor is defined, and use the logical TensorFlow ops to test the value of a tensor.

In [48]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (1 * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = training_data.next_batch(batch_size)
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data}
    _, l, feature_weights = session.run(
      [optimizer, loss, weights_hidden1], feed_dict=feed_dict)
    if (step % 500 == 0):
        for i in xrange(batch_size):
            plt.imshow(feature_weights[i].reshape(50,50), cmap=cm.gray)
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized


ValueError: could not broadcast input array from shape (128,784) into shape (128)

---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---


## With just two additional layers

In [67]:
batch_size = 128
nHidden1=1024
nHidden2=300
nHidden3=50
beta=.0001

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights_hidden1 = tf.Variable(
    tf.truncated_normal([image_size * image_size, nHidden1]))
  weights_hidden2 = tf.Variable(
    tf.truncated_normal([nHidden1, nHidden2])) 
  weights_hidden3 = tf.Variable(
    tf.truncated_normal([nHidden2, nHidden3]))
  weights = tf.Variable(
    tf.truncated_normal([nHidden3, num_labels]))
  biases_hidden1 = tf.Variable(tf.zeros([nHidden1]))
  biases_hidden2 = tf.Variable(tf.zeros([nHidden2]))
  biases_hidden3 = tf.Variable(tf.zeros([nHidden3]))
  biases = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  hidden1 = tf.nn.dropout(tf.nn.relu(tf.matmul(tf_train_dataset, weights_hidden1) + biases_hidden1), 0.5 )
  hidden2 = tf.nn.dropout(tf.nn.relu(tf.matmul(hidden1, weights_hidden2) + biases_hidden2), 0.5 )
  hidden3 = tf.nn.dropout(tf.nn.relu(tf.matmul(hidden2, weights_hidden3) + biases_hidden3), 0.5 )
  logits = tf.matmul(hidden3, weights) + biases
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))+beta*(
    tf.nn.l2_loss(weights_hidden1)+tf.nn.l2_loss(weights_hidden2)+tf.nn.l2_loss(weights_hidden3))
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.0005).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  
  valid_hLayer1=tf.nn.relu(tf.matmul(tf_valid_dataset, weights_hidden1) + biases_hidden1)
  valid_hLayer2=tf.nn.relu(tf.matmul(valid_hLayer1, weights_hidden2) + biases_hidden2)
  valid_hLayer3=tf.nn.relu(tf.matmul(valid_hLayer2, weights_hidden3) + biases_hidden3)
  valid_prediction = tf.nn.softmax(tf.matmul(valid_hLayer3, weights) + biases)

  test_hLayer1=tf.nn.relu(tf.matmul(tf_test_dataset, weights_hidden1) + biases_hidden1)
  test_hLayer2=tf.nn.relu(tf.matmul(test_hLayer1, weights_hidden2) + biases_hidden2)
  test_hLayer3=tf.nn.relu(tf.matmul(test_hLayer2, weights_hidden3) + biases_hidden3)
  test_prediction = tf.nn.softmax(tf.matmul(test_hLayer3, weights) + biases)
  

In [68]:
num_steps = 5001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
        plt.imshow(weights.reshape(50,50), cmap=cm.gray)
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 48924.171875
Minibatch accuracy: 8.6%
Validation accuracy: 13.9%
Minibatch loss at step 500: 62.357037
Minibatch accuracy: 10.9%
Validation accuracy: 10.0%
Minibatch loss at step 1000: 62.448425
Minibatch accuracy: 9.4%
Validation accuracy: 10.0%
Minibatch loss at step 1500: 50.824585
Minibatch accuracy: 8.6%
Validation accuracy: 10.0%
Minibatch loss at step 2000: 51.878212
Minibatch accuracy: 10.9%
Validation accuracy: 10.0%
Minibatch loss at step 2500: 46.911125
Minibatch accuracy: 8.6%
Validation accuracy: 10.0%
Minibatch loss at step 3000: 47.891376
Minibatch accuracy: 9.4%
Validation accuracy: 10.0%
Minibatch loss at step 3500: 45.857243
Minibatch accuracy: 10.9%
Validation accuracy: 10.0%
Minibatch loss at step 4000: 46.066219
Minibatch accuracy: 7.8%
Validation accuracy: 10.0%
Minibatch loss at step 4500: 45.710533
Minibatch accuracy: 9.4%
Validation accuracy: 10.0%
Minibatch loss at step 5000: 46.827404
Minibatch accuracy: 11.7%
Validation 

## With dynamic learning rate

In [83]:
tf.train.exponential_decay?

In [87]:
batch_size = 128
nHidden1=1024
nHidden2=300
nHidden3=50
beta=.00001

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  global_step = tf.Variable(0)  # count the number of steps taken.


  # Variables.
  weights_hidden1 = tf.Variable(
    tf.truncated_normal([image_size * image_size, nHidden1]))
  weights_hidden2 = tf.Variable(
    tf.truncated_normal([nHidden1, nHidden2])) 
  weights_hidden3 = tf.Variable(
    tf.truncated_normal([nHidden2, nHidden3]))
  weights = tf.Variable(
    tf.truncated_normal([nHidden3, num_labels]))
  biases_hidden1 = tf.Variable(tf.zeros([nHidden1]))
  biases_hidden2 = tf.Variable(tf.zeros([nHidden2]))
  biases_hidden3 = tf.Variable(tf.zeros([nHidden3]))
  biases = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  hidden1 = tf.nn.dropout(tf.nn.relu(tf.matmul(tf_train_dataset, weights_hidden1) + biases_hidden1), 0.5 )
  hidden2 = tf.nn.dropout(tf.nn.relu(tf.matmul(hidden1, weights_hidden2) + biases_hidden2), 0.5 )
  hidden3 = tf.nn.dropout(tf.nn.relu(tf.matmul(hidden2, weights_hidden3) + biases_hidden3), 0.5 )
  logits = tf.matmul(hidden3, weights) + biases
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))+beta*(
    tf.nn.l2_loss(weights_hidden1)+tf.nn.l2_loss(weights_hidden2)+tf.nn.l2_loss(weights_hidden3))
  learning_rate = tf.train.exponential_decay(0.0001, global_step, decay_steps=2000,decay_rate=.01)
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  
  valid_hLayer1=tf.nn.relu(tf.matmul(tf_valid_dataset, weights_hidden1) + biases_hidden1)
  valid_hLayer2=tf.nn.relu(tf.matmul(valid_hLayer1, weights_hidden2) + biases_hidden2)
  valid_hLayer3=tf.nn.relu(tf.matmul(valid_hLayer2, weights_hidden3) + biases_hidden3)
  valid_prediction = tf.nn.softmax(tf.matmul(valid_hLayer3, weights) + biases)

  test_hLayer1=tf.nn.relu(tf.matmul(tf_test_dataset, weights_hidden1) + biases_hidden1)
  test_hLayer2=tf.nn.relu(tf.matmul(test_hLayer1, weights_hidden2) + biases_hidden2)
  test_hLayer3=tf.nn.relu(tf.matmul(test_hLayer2, weights_hidden3) + biases_hidden3)
  test_prediction = tf.nn.softmax(tf.matmul(test_hLayer3, weights) + biases)
  

In [88]:
num_steps = 20001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 48858.390625
Minibatch accuracy: 7.8%
Validation accuracy: 10.3%
Minibatch loss at step 500: 363.485962
Minibatch accuracy: 13.3%
Validation accuracy: 10.4%
Minibatch loss at step 1000: 187.224503
Minibatch accuracy: 14.1%
Validation accuracy: 14.4%
Minibatch loss at step 1500: 336.029419
Minibatch accuracy: 10.9%
Validation accuracy: 11.5%
Minibatch loss at step 2000: 122.034492
Minibatch accuracy: 10.2%
Validation accuracy: 11.9%
Minibatch loss at step 2500: 85.096031
Minibatch accuracy: 11.7%
Validation accuracy: 17.1%
Minibatch loss at step 3000: 144.306717
Minibatch accuracy: 8.6%
Validation accuracy: 23.2%
Minibatch loss at step 3500: 99.710976
Minibatch accuracy: 15.6%
Validation accuracy: 25.1%
Minibatch loss at step 4000: 124.454926
Minibatch accuracy: 14.1%
Validation accuracy: 26.3%
Minibatch loss at step 4500: 171.098618
Minibatch accuracy: 12.5%
Validation accuracy: 26.8%
Minibatch loss at step 5000: 187.623260
Minibatch accuracy: 13.3